<a href="https://colab.research.google.com/github/danielbauer1979/ML_656/blob/main/Module6_CART_Tutorial_upd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Classification and Regression Trees (CART)

Dani Bauer, 2022

In this tutorial, we will introduce classification and regression trees.  We will start with a basic simulated example with a single predictor that compares tress to (linear) regression.  In particular, we will demonstrate when trees do well and when the don't.  Subsequently, we will revisit our Caravan case study.

As usually, let's start with loading the relevant libaries.

In [1]:
import numpy as np 
import matplotlib.pyplot as plt  
import pandas as pd 
import seaborn as sns
import graphviz
import pydot
from io import StringIO  

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn.metrics import mean_squared_error,confusion_matrix, classification_report

# Regression Trees

## Review of Concepts and Maths

### Trees

As we discuss in the lecture, trees successively split the domains of predictors into two areas (greater and smaller), so that the final tree can be thought of as a model with constant predictions in *square partitions* of the feature domain.  Hence, in building a tree, we need to address two questions:

1. <span style="color:blue"> How to come up with the splits that determine the square partitions? </span>

2. <span style="color:blue"> How to come up with the predictions for each of the square partitions? </span>

The second task is straightforward:  We simply take the average of all predictions in the current partition.  

For 2., one relies on a *greedy algorithm*: For each node, we go over all possible variables and all possible split levels, and then choose the combination of variable and split level that mimizes the prediction error.  However, note that the tree thus will generally not minimize the squared prediction error -- finding such a partition is computationally/algorithmically infeasible...

### Excursion: The Sigmoid Function

In generating our dataset we will use the so-called *sigmoid function*, which will be a key ingredient when we discuss *neural networks*.  It has the advantage that depending on the choice of a parameter, it can mimic a highly linear or a highly non-linear relationship:

$$
\sigma_{\alpha}(x) = \frac{1}{1+e^{-\alpha\,x}}
$$


In [ ]:
def sigmoid(x):
    return(1 / (1 + np.exp(-x)))
x = np.linspace(-5, 5, 1000)
plt.figure(figsize=(15,6))
plt.plot(x, sigmoid(x), color='b')
plt.plot(x, sigmoid(0.5*x), color='r')
plt.plot(x, sigmoid(5*x), color='g')

So we notice that for a small choice of $\alpha$ (red curve), we obtain a relationship that is almost linear (and, indeed, as $\alpha$ becomes smaller and smaller, $\sigma_{\alpha}$ approaches a perfectly linear function).  Whereas for a larger choice of $\alpha$ (green curve), we almost obtain a step function (and, indeed, as $\alpha$ becomes larger and larger, $\sigma_{\alpha}$ approaches a step function).

## A Simulated Example with One Predictor

1. Let's start by generating two datasets based on the sigmoid function with different parameters taken from above. 

In [ ]:
np.random.seed(1)
x = 3 * np.random.normal(0, 1, 150)
eps = 0.25 * np.random.normal(0, 1, 150)

y_1 = sigmoid(0.5 * x) + eps
y_2 = sigmoid(5 * x) + eps

mydata1 = pd.DataFrame({'y_1':y_1,'x':x})
mytraindata1 = mydata1[1:100]
mytestdata1 = mydata1[101:150]

plt.scatter(mydata1.x,mydata1.y_1)

In [ ]:
mydata2 = pd.DataFrame({'y_2':y_2,'x':x})
mytraindata2 = mydata2[1:100]
mytestdata2 = mydata2[101:150]
plt.scatter(mydata2.x,mydata2.y_2)

2. And, for comparison, let's fit a linear regression model to both datasets and let's evaluate the out-of-sample error. 

In [ ]:
X_1train = mytraindata1['x'].values.reshape(-1,1)
X_1test = mytestdata1['x'].values.reshape(-1,1)
y_1train = mytraindata1['y_1'].values
lmfit1 = LinearRegression(fit_intercept=True)
lmfit1.fit(X_1train,y_1train)
yhat_OOS1 = lmfit1.predict(X_1test)
OLS_OOS_MSE1 = sum((mytestdata1['y_1'] - yhat_OOS1)**2)/50
OLS_OOS_MSE1

In [ ]:
X_2train = mytraindata2['x'].values.reshape(-1,1)
X_2test = mytestdata2['x'].values.reshape(-1,1)
y_2train = mytraindata2['y_2'].values
lmfit2 = LinearRegression(fit_intercept=True)
lmfit2.fit(X_2train,y_2train)
yhat_OOS2 = lmfit2.predict(X_2test)
OLS_OOS_MSE2 = sum((mytestdata2['y_2'] - yhat_OOS2)**2)/50
OLS_OOS_MSE2

3. Let's fit a tree to the second dataset. 

In [ ]:
tree2 = DecisionTreeRegressor(max_leaf_nodes=3)
tree2.fit(X_2train, y_2train)

The following function creates images of tree models using pydot, as the package sklearn doesn't offer graphs of the trees


In [8]:
import pydot
from IPython.display import Image
def print_tree(estimator, features, class_names=None, filled=True):
    tree = estimator
    names = features
    color = filled
    classn = class_names
    
    dot_data = StringIO()
    export_graphviz(estimator, out_file=dot_data, feature_names=features, class_names=classn, filled=filled)
    graph = pydot.graph_from_dot_data(dot_data.getvalue())
    return(graph)

In [ ]:
graph, = print_tree(tree2, features=['x'])
Image(graph.create_png())

Note: Tree algorithms are implemented differently in R and Python, so generally we don't expect exactly the same results from R and Python.

4. Let's use cross validation to see what size is optimal, and let's prune accordingly.

In [ ]:
from sklearn.model_selection import cross_val_score
OLS_OOS_MSE2 = []
cv_score = []
for i in range(2,7):
    tree2 = DecisionTreeRegressor(max_leaf_nodes=i)
    tree2.fit(X_2train, y_2train)
    ytreehat2 = tree2.predict(X_2test)
    OLS_OOS_MSE2.append(sum((mytestdata2['y_2'] - ytreehat2)**2)/50)
    cv_score.append(sum(cross_val_score(tree2, X_2train, y_2train, cv=5)))
plt.plot(range(2,7),OLS_OOS_MSE2)

In [ ]:
plt.plot(range(2,7),cv_score)

Python doesn't have automatic pruning algorithm. We can prune ourselves by cross validation with hyper-parameters in the DecisionTreeRegressor function. 
More info in the  [scikit-docu](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

5. Let's redo for the first dataset.

In [ ]:
tree1 = DecisionTreeRegressor()
tree1.fit(X_1train, y_1train)
graph, = print_tree(tree1, features=['x'])
Image(graph.create_png())

In [ ]:
OLS_OOS_MSE1 = []
cv_score = []
for i in range(2,7):
    tree1 = DecisionTreeRegressor(max_leaf_nodes=i)
    tree1.fit(X_1train, y_1train)
    ytreehat1 = tree1.predict(X_1test)
    OLS_OOS_MSE1.append(sum((mytestdata1['y_1'] - ytreehat1)**2)/50)
    cv_score.append(sum(cross_val_score(tree1, X_1train, y_1train, cv=5)))
plt.plot(range(2,7),OLS_OOS_MSE1)

In [ ]:
plt.plot(range(2,7),cv_score)

Hence, the tree performs very differently between the two datasets:  For the second dataset, the tree provided a very simple (and understandable!) model of the data, and despite the simplicity the prediction accuracy improved relative to the linear regressions.  For the first dataset, on the other hand, the tree is far more complex than the regression model but still provides worse predictions.

Hence, overall, trees are well-suited for modeling non-linear relationships -- which explains their relevance as the foundational structural element for more advanced learners.

## Case Study I: Caravan Insurance Purchases

Let's go back to the `Caravan` insurance data:

In [ ]:
!git clone https://github.com/danielbauer1979/ML_656.git

In [16]:
Caravan = pd.read_csv('ML_656/Caravan.csv', index_col=0)

As a reminder, our GLM and GAM models did not get a single true-positive with a cutoff of 50% -- essentially the predictions were all zero.  We obtained an AUC of 0.7456 for the GAM.  The knn classifier did better.  While we cannot obtain an AUC, particularlty for $k=1$ we obtained 45 true classifications, and 10 were correct.  So that makes for a true-postitive rate of roughly 18%.

### Classification Tree

Let's try a classification tree, with the caveat that we have to change the default parameters. The standard value of the so-called complexity parameter `cp` is insufficient to generate sufficient splits, because a split only happens if there is sufficient heterogeneity in the nodes.  We set it to 0.001 but we can generate an even larger tree by a lower choice.

In [ ]:
Caravan.Purchase = Caravan.Purchase=='Yes'
test = Caravan.iloc[0:1000]
train = Caravan.iloc[1000:len(Caravan)]
X = train.drop(columns = ['Purchase'])
y = train.Purchase
Car_tree_first = DecisionTreeRegressor(max_leaf_nodes=4)
Car_tree_first.fit(X, y)
graph, = print_tree(Car_tree_first, features=X.columns)
Image(graph.create_png())

The issue with growing the tree is that there are few positives, leading to substantial "note purity" even after a few modeling steps. We have to adjust the parameters to build a larger tree:

In [ ]:
Car_tree = DecisionTreeRegressor(min_samples_split=5,min_impurity_decrease=0.0001)
Car_tree.fit(X, y)
graph, = print_tree(Car_tree, features=X.columns)
Image(graph.create_png())

Let's look at the top features by importance scores:

In [ ]:
summary_tree = pd.DataFrame({'Features':X.columns,'Importance':Car_tree.feature_importances_}) 
summary_tree.sort_values(by=['Importance'], ascending=False)[0:10]

The tree has the following final nodes:

In [ ]:
Car_tree.tree_.node_count

Let's look at the number of "purchases":

In [ ]:
yhat = Car_tree.predict(test.drop(columns = ['Purchase']))
pred = yhat > 0.5
np.sum(pred)

And the confusion table is:

In [ ]:
table = pd.DataFrame({'Purchase':test.Purchase,'pred':pred})
table.groupby(['Purchase','pred']).size().unstack('Purchase')

Much better!